In [36]:
import pandas as pd
import numpy as np
import os


In [37]:
df = pd.read_csv(os.path.join(os.getcwd(), "sars_data.csv"))
df.head()


,Date,Country,Cumulative number of case(s),Number of deaths,Number recovered
0,2003-03-17,Germany,1,0,0
1,2003-03-17,Canada,8,2,0
2,2003-03-17,Singapore,20,0,0
3,2003-03-17,"Hong Kong SAR, China",95,1,0
4,2003-03-17,Switzerland,2,0,0


In [38]:
df["Country"].value_counts()

def country_western_europe(df):
    if (df['Country'] == 'France') or (df['Country'] == 'Germany') or (df['Country'] == 'United Kingdom') or (df['Country'] == 'Republic of Ireland ') or (df['Country'] == 'Italy') or (df['Country'] == 'Spain'):
        return 1
    else:
        return 0

def country_eastern_asia(df):
    if (df['Country'] == 'China') or (df['Country'] == 'Hong Kong SAR, China') or (df['Country'] == 'Taiwan, China') or (df['Country'] == 'Republic of Korea') or (df['Country'] == 'Macao SAR, China'):
        return 1
    else:
        return 0

def country_south_east_asia(df):
    if (df['Country'] == 'Thailand') or (df['Country'] == 'Singapore') or (df['Country'] == 'Viet Nam') or (df['Country'] == 'Malaysia') or (df['Country'] == 'Indonesia') or (df['Country'] == 'Philippines'):
        return 1
    else:
        return 0


df['is_western_europe'] = df.apply(country_western_europe, axis=1)
df['is_east_asian'] = df.apply(country_eastern_asia, axis=1)
df['is_south_east_asian'] = df.apply(country_south_east_asia, axis=1)
df.drop(df[(df['is_western_europe'] == 0) & (df['is_east_asian'] == 0) & (df['is_south_east_asian'] == 0)].index , inplace=True)
df.head()

,Date,Country,Cumulative number of case(s),Number of deaths,Number recovered,is_western_europe,is_east_asian,is_south_east_asian
0,2003-03-17,Germany,1,0,0,1,0,0
2,2003-03-17,Singapore,20,0,0,0,0,1
3,2003-03-17,"Hong Kong SAR, China",95,1,0,0,1,0
5,2003-03-17,Thailand,1,0,0,0,0,1
6,2003-03-17,Viet Nam,40,1,0,0,0,1


In [39]:
def reverse_label_encode(df):
    if (df['is_western_europe'] == 1):
        return "western_europe"
    elif(df['is_east_asian'] == 1):
        return "east_asian"
    else:
        return "south_east_asian"
    
df['region'] = df.apply(reverse_label_encode, axis=1)
df.drop(['is_western_europe', 'is_east_asian', 'is_south_east_asian'], axis=1, inplace=True)
df.head()   

,Date,Country,Cumulative number of case(s),Number of deaths,Number recovered,region
0,2003-03-17,Germany,1,0,0,western_europe
2,2003-03-17,Singapore,20,0,0,south_east_asian
3,2003-03-17,"Hong Kong SAR, China",95,1,0,east_asian
5,2003-03-17,Thailand,1,0,0,south_east_asian
6,2003-03-17,Viet Nam,40,1,0,south_east_asian


In [40]:
df["region"].value_counts()

df['cases'] = df.groupby(['Date', 'region'])[['Cumulative number of case(s)']].transform('sum').sum(1)
df['deaths'] = df.groupby(['Date', 'region'])[['Number of deaths']].transform('sum').sum(1)
df['recoveries'] = df.groupby(['Date', 'region'])[['Number recovered']].transform('sum').sum(1)
df.drop(['Cumulative number of case(s)', 'Number of deaths', 'Number recovered'], axis=1, inplace=True)


In [41]:
df.head()

,Date,Country,region,cases,deaths,recoveries
0,2003-03-17,Germany,western_europe,1,0,0
2,2003-03-17,Singapore,south_east_asian,61,1,0
3,2003-03-17,"Hong Kong SAR, China",east_asian,95,1,0
5,2003-03-17,Thailand,south_east_asian,61,1,0
6,2003-03-17,Viet Nam,south_east_asian,61,1,0


In [45]:
df.sort_values(['Date', 'region'], inplace = True, ascending=[True, False])
df['delta cases'] = (df.groupby('Date')['cases'].apply(pd.Series.diff) + 1)
df['delta deaths'] = (df.groupby('Date')['deaths'].apply(pd.Series.diff) + 1)
df['delta recoveries'] = (df.groupby('Date')['recoveries'].apply(pd.Series.diff) + 1)

In [49]:
df = df.fillna(0)
df.head()

,Date,Country,region,cases,deaths,recoveries,delta cases,delta deaths,delta recoveries
0,2003-03-17,Germany,western_europe,1,0,0,0.0,0.0,0.0
2,2003-03-17,Singapore,south_east_asian,61,1,0,61.0,2.0,1.0
5,2003-03-17,Thailand,south_east_asian,61,1,0,1.0,1.0,1.0
6,2003-03-17,Viet Nam,south_east_asian,61,1,0,1.0,1.0,1.0
3,2003-03-17,"Hong Kong SAR, China",east_asian,95,1,0,35.0,1.0,1.0


In [50]:
df.to_csv(os.path.join(os.getcwd(), "cleaned_sars_data.csv"), index=False)
